In [1]:
import math
import metapy
import pytoml
import csv
import pandas as pd


In [3]:
with open('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_segments.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(data[0])

['key', 'course_id', 'week_nbr', 'video_id', 'video_title', 'timeline_start', 'timeline_end', 'segment_nbr', 'segment_link', 'segment_txt']


In [4]:

df = pd.read_csv ('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_segments.csv')

In [5]:
print(df.columns)

Index(['key', 'course_id', 'week_nbr', 'video_id', 'video_title',
       'timeline_start', 'timeline_end', 'segment_nbr', 'segment_link',
       'segment_txt'],
      dtype='object')


In [6]:
segcolumns = ['course_id','segment_txt']

In [7]:
new_df_corse_text = pd.DataFrame(df, columns=segcolumns)

In [8]:
print(new_df_corse_text)

      course_id                                        segment_txt
0        cs-410                                            [SOUND]
1        cs-410  Hello welcome to CS410 DSO Text Information Sy...
2        cs-410          >> This lecture is about Natural Language
3        cs-410  This is an online course offered by University...
4        cs-410                               of Content Analysis.
...         ...                                                ...
12784    cs-410                So I hope you can use what you have
12785    cs-410              applications will benefit society and
12786    cs-410             the research community to discover new
12787    cs-410                                         Thank you.
12788    cs-410                                            [MUSIC]

[12789 rows x 2 columns]


In [9]:
textcolumn = ['segment_txt']
new_df_text = pd.DataFrame(df, columns=textcolumn)

In [10]:
print(new_df_text)

                                             segment_txt
0                                                [SOUND]
1      Hello welcome to CS410 DSO Text Information Sy...
2              >> This lecture is about Natural Language
3      This is an online course offered by University...
4                                   of Content Analysis.
...                                                  ...
12784                So I hope you can use what you have
12785              applications will benefit society and
12786             the research community to discover new
12787                                         Thank you.
12788                                            [MUSIC]

[12789 rows x 1 columns]


In [11]:
corpus = []
for index, row in df.iterrows():
    text = row['segment_txt']
    corpus.append(text)

In [12]:
with open('course.dat', 'w') as f:
    for text in corpus:
        f.write(text)
        f.write('\n')

In [13]:
idx = metapy.index.make_inverted_index('config.toml')

In [14]:
idx.num_docs()

12502

In [15]:
idx.unique_terms()

1987

In [16]:
idx.avg_doc_length()

5.623100280761719

In [17]:
idx.total_corpus_terms()

70300

In [18]:
ranker = metapy.index.OkapiBM25()

In [19]:
query = metapy.index.Document()
query.content('What is PLSA?') 

In [20]:
top_docs = ranker.score(idx, query, num_results=5)
top_docs

[(6043, 9.34522819519043),
 (6082, 9.238581657409668),
 (820, 9.238581657409668),
 (753, 7.36001443862915),
 (2331, 7.36001443862915)]

In [21]:
for num, (d_id, _) in enumerate(top_docs):
    print(corpus[d_id])

is you will find the new
layout, etc.
to do well in the quizzes and exams,
So, we would have to give those issues the highest priority,
And we generally, need to use a lot of


In [22]:
for num, (d_id, _) in enumerate(top_docs):
    print(df.loc[d_id].segment_link)

https://www.coursera.org/learn/cs-410/lecture/qkTHD?t=439
https://www.coursera.org/learn/cs-410/lecture/qkTHD?t=615
https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=2264
https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=1961
https://www.coursera.org/learn/cs-410/lecture/RnXhr?t=985


In [25]:
for num, (d_id, _) in enumerate(top_docs):
    print(df.loc[d_id].segment_link)

https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=18
https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=162
https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=407
https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=112
https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=330


In [23]:
import math
import metapy
import pytoml
import csv
import pandas as pd


class SearchEngine:
    def __init__(self, path, config):
        self.path = path
        self.idx = metapy.index.make_inverted_index(config)
        self.config = config
        self.query = metapy.index.Document()
        self.ranker = self.ranker(self.config)
        
    def build_corpus(self):
        df = pd.read_csv(self.path)
        textcolumn = ['segment_txt'] # ignore the other columns
        new_df_text = pd.DataFrame(df, columns=textcolumn)
        with open('course.dat', 'w') as f:
            for index, row in df.iterrows():
                text = row['segment_txt']
                f.write(text)
                f.write('\n')
        return df

    def ranker(self, config):
         return metapy.index.OkapiBM25(k1=1.65, b=0.75, k3=2.6)
        
    def ranker_score(self):
        ranker = self.ranker
        top_docs = ranker.score(self.idx, self.query, num_results=5)
        return top_docs

    def query_result(self,text):
        self.query.content(text) 
        top_docs = self.ranker_score()
        df = self.build_corpus()
        top_links = []
        for num, (d_id, _) in enumerate(top_docs):
            top_links.append(df.loc[d_id].segment_link)
        return top_links

In [24]:
q1 = SearchEngine('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_segments.csv', 'config.toml')
print(q1.query_result('What is PLSA?'))

['https://www.coursera.org/learn/cs-410/lecture/qkTHD?t=439', 'https://www.coursera.org/learn/cs-410/lecture/qkTHD?t=615', 'https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=2264', 'https://www.coursera.org/learn/cs-410/lecture/rLpwp?t=1961', 'https://www.coursera.org/learn/cs-410/lecture/RnXhr?t=985']
